In [1]:
!pip install autogluon openml

In [2]:
import time
import pandas as pd
import numpy as np
import openml
import os
import math # Importa a biblioteca de matemática para o cálculo do teto

from autogluon.tabular import TabularPredictor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

# Configurações
SEED = 42

def carregar_base_openml(openml_id):
    """Carrega um dataset do OpenML e prepara os tipos de dados."""
    dataset = openml.datasets.get_dataset(openml_id, download_data=True, download_qualities=True,
                                          download_features_meta_data=True)
    X, y, _, attribute_names = dataset.get_data(dataset_format="dataframe", target=dataset.default_target_attribute)

    if isinstance(y, pd.Series):
        y = y.values.ravel()

    categorical_cols_names = [name for i, name in enumerate(attribute_names) if
                              dataset.features[i].data_type == 'nominal']
    for col_name in categorical_cols_names:
        if col_name in X.columns:
            X[col_name] = X[col_name].astype('category')

    return X, y


In [3]:
def run_autogluon_experiment(openml_id):
    """
    Executa o fluxo completo de avaliação para o AutoGluon em um dataset.
    """
    print(f"--- Processando Dataset ID: {openml_id} com AutoGluon ---")
    
    AUTOML_TIME_BUDGET = 3600

    # 1. Carregamento e pré-processamento dos dados
    X_df, y_orig = carregar_base_openml(openml_id)

    X_train_df, X_test_df, y_train_orig, y_test_orig = train_test_split(
        X_df, y_orig, test_size=0.3, random_state=SEED, stratify=(y_orig if len(np.unique(y_orig)) > 1 else None)
    )

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_orig)
    y_test_enc = le.transform(y_test_orig)
    
    # Lógica de seleção de métrica dinâmica
    n_classes = len(le.classes_)
    if n_classes > 2:
        metric_to_use = 'roc_auc_ovo'
        print(f"Problema multiclasse detectado ({n_classes} classes). Usando a métrica: {metric_to_use}")
    else:
        metric_to_use = 'roc_auc'
        print(f"Problema binário detectado. Usando a métrica: {metric_to_use}")

    # Imputação de dados faltantes
    num_cols = X_train_df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0 and X_train_df[num_cols].isnull().sum().sum() > 0:
        imp = SimpleImputer(strategy='median')
        X_train_df.loc[:, num_cols] = imp.fit_transform(X_train_df[num_cols])
        X_test_df.loc[:, num_cols] = imp.transform(X_test_df[num_cols])

    cat_cols = X_train_df.select_dtypes(include=['category', 'object']).columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            X_train_df.loc[:, col] = X_train_df[col].astype(str).fillna("__MISSING__")
            X_test_df.loc[:, col] = X_test_df[col].astype(str).fillna("__MISSING__")

    # 2. Treinamento com AutoGluon
    t0_total = time.time()

    train_df_ag = X_train_df.copy()
    target_col = 'target'
    train_df_ag[target_col] = y_train_enc
    
    save_path = f'autogluon_models/ds_{openml_id}'

    predictor = TabularPredictor(
        label=target_col,
        path=save_path,
        eval_metric=metric_to_use
    ).fit(
        train_data=train_df_ag,
        time_limit=AUTOML_TIME_BUDGET,
        presets='best_quality'
    )

    leaderboard = predictor.leaderboard(silent=True)
    best_model_name = leaderboard.iloc[0]['model']
    best_model_score_val = leaderboard.iloc[0]['score_val']

    # 3. Predição e Métricas
    y_proba = predictor.predict_proba(X_test_df, as_multiclass=True).values
    y_pred = predictor.predict(X_test_df).values
    
    total_time = time.time() - t0_total

    if n_classes == 2:
        auc_ovo = roc_auc_score(y_test_enc, y_proba[:, 1])
    else:
        auc_ovo = roc_auc_score(y_test_enc, y_proba, multi_class='ovo')

    acc = accuracy_score(y_test_enc, y_pred)
    ce = log_loss(y_test_enc, y_proba, labels=le.transform(le.classes_))

    print(f"Resultados para {openml_id}: AUC={auc_ovo:.4f}, ACC={acc:.4f}, Time={total_time:.2f}s")
    
    return {
        'openml_id': openml_id, 'modelo': 'autogluon', 'total_time_sec': total_time,
        'mean_auc_ovo': auc_ovo, 'mean_accuracy': acc, 'mean_cross_entropy': ce,
        'best_model_info': best_model_name,
        'best_score_cv': best_model_score_val
    }

In [4]:
if __name__ == "__main__":
    # --- CONFIGURAÇÃO DA EXECUÇÃO DISTRIBUÍDA ---
    # Altere esta variável em cada máquina de 1 a 10
    MACHINE_ID = 1
    TOTAL_MACHINES = 10
    # --- FIM DA CONFIGURAÇÃO ---

    # Lista completa dos 30 datasets do CC18
    cc18_ids_full = [
        11, 15, 18, 23, 29, 31, 37, 50, 54, 188, 307, 458, 469, 1049,
        1050, 1063, 1068, 1462, 1464, 1468, 1480, 1494, 1501, 1510,
        6332, 23381, 40966, 40975, 40982, 40994
    ]

    # Lógica para dividir a lista de IDs em lotes
    chunk_size = math.ceil(len(cc18_ids_full) / TOTAL_MACHINES)
    start_index = (MACHINE_ID - 1) * chunk_size
    end_index = start_index + chunk_size
    ids_to_process = cc18_ids_full[start_index:end_index]

    print(f"--- MÁQUINA {MACHINE_ID}/{TOTAL_MACHINES} ---")
    print(f"Processando os seguintes datasets: {ids_to_process}")
    
    todos_resultados = []

    for oid in ids_to_process:
        try:
            res = run_autogluon_experiment(oid)
            todos_resultados.append(res)
        except Exception as e:
            print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
            import traceback
            traceback.print_exc()
            continue

    # Salva o arquivo com um nome único para esta máquina
    if todos_resultados:
        df_results = pd.DataFrame(todos_resultados)
        output_filename = f"resultados_autogluon_maquina_{MACHINE_ID}.csv"
        df_results.to_csv(output_filename, index=False)
        print(f"\nArquivo '{output_filename}' salvo com sucesso!")
    else:
        print("Nenhuma tarefa foi concluída com sucesso nesta máquina.")


--- MÁQUINA 1/10 ---
Processando os seguintes datasets: [11, 15, 18]
--- Processando Dataset ID: 11 com AutoGluon ---


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          8
Memory Avail:       29.98 GB / 31.36 GB (95.6%)
Disk Space Avail:   94.43 GB / 97.87 GB (96.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.

Problema multiclasse detectado (3 classes). Usando a métrica: roc_auc_ovo


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-06-14 10:15:52,463	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
		Context path: "/home/jupyter/autogluon_models/ds_11/ds_sub_fit/sub_fit_ho"
(_dystack pid=4505) Running DyStack sub-fit ...
(_dystack pid=4505) Beginning AutoGluon training ... Time limit = 895s
(_dystack pid=4505) AutoGluon will save models to "/home/jupyter/autogluon_models/ds_11/ds_sub_fit/sub_fit_ho"
(_dystack pid=4505) Train Data Rows:    388
(_dystack pid=4505) Train Data Columns: 4
(_dystack pid=4505) Label Column:       target
(_dystack pid=4505) Problem Type:       multiclass
(_dystack pid=4505) Preprocessing data ...
(_dystack pid=4505) Train Data Class Count: 3
(_dystack pid=4505) Using Feature Generators to preprocess the data ...
(_dystack pid=4505) Fitting AutoMLPi

(_ray_fit pid=5263) [1000]	valid_set's multi_logloss: 0.300361	valid_set's roc_auc_ovo: 0.952569


...

Resultados para 15: AUC=0.9904, ACC=0.9476, Time=1706.53s
--- Processando Dataset ID: 18 com AutoGluon ---


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          8
Memory Avail:       28.03 GB / 31.36 GB (89.4%)
Disk Space Avail:   92.00 GB / 97.87 GB (94.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.

Problema multiclasse detectado (10 classes). Usando a métrica: roc_auc_ovo


Leaderboard on holdout data (DyStack):
                          model  score_holdout  score_val  eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         ExtraTreesEntr_BAG_L2       0.978418   0.972752  roc_auc_ovo        3.297737       1.758525  338.014614                 0.189690                0.128654           1.263488            2       True         29
1           WeightedEnsemble_L3       0.977830   0.977830  roc_auc_ovo        3.633031       2.111887  492.972016                 0.004455                0.043316          18.305284            3       True         31
2       RandomForestEntr_BAG_L2       0.977633   0.971720  roc_auc_ovo        3.296243       1.749718  338.042793                 0.188196                0.119847           1.291667            2       True         26
3               CatBoost_BAG_L2       0.977605   0.973566  roc_auc_ovo        3.154747       

Resultados para 18: AUC=0.9779, ACC=0.7867, Time=3631.28s

Arquivo 'resultados_autogluon_maquina_1.csv' salvo com sucesso!


In [6]:
# --- CONFIGURAÇÃO DA EXECUÇÃO DISTRIBUÍDA ---
# Altere esta variável em cada máquina de 1 a 10
MACHINE_ID = 6
TOTAL_MACHINES = 10
# --- FIM DA CONFIGURAÇÃO ---

# Lista completa dos 30 datasets do CC18
cc18_ids_full = [
    11, 15, 18, 23, 29, 31, 37, 50, 54, 188, 307, 458, 469, 1049,
    1050, 1063, 1068, 1462, 1464, 1468, 1480, 1494, 1501, 1510,
    6332, 23381, 40966, 40975, 40982, 40994
]

# Lógica para dividir a lista de IDs em lotes
chunk_size = math.ceil(len(cc18_ids_full) / TOTAL_MACHINES)
start_index = (MACHINE_ID - 1) * chunk_size
end_index = start_index + chunk_size
ids_to_process = cc18_ids_full[start_index:end_index]

print(f"--- MÁQUINA {MACHINE_ID}/{TOTAL_MACHINES} ---")
print(f"Processando os seguintes datasets: {ids_to_process}")

todos_resultados = []

for oid in ids_to_process:
    try:
        res = run_autogluon_experiment(oid)
        todos_resultados.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

# Salva o arquivo com um nome único para esta máquina
if todos_resultados:
    df_results = pd.DataFrame(todos_resultados)
    output_filename = f"resultados_autogluon_maquina_{MACHINE_ID}.csv"
    df_results.to_csv(output_filename, index=False)
    print(f"\nArquivo '{output_filename}' salvo com sucesso!")
else:
    print("Nenhuma tarefa foi concluída com sucesso nesta máquina.")


--- MÁQUINA 6/10 ---
Processando os seguintes datasets: [1063, 1068, 1462]
--- Processando Dataset ID: 1063 com AutoGluon ---


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          8
Memory Avail:       27.97 GB / 31.36 GB (89.2%)
Disk Space Avail:   90.45 GB / 97.87 GB (92.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.

Problema binário detectado. Usando a métrica: roc_auc


Leaderboard on holdout data (DyStack):
                           model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetTorch_r30_BAG_L2       0.804924   0.898020     roc_auc        1.065310       0.944868  32.116456                 0.256950                0.216911           8.718235            2       True        127
1           LightGBM_r143_BAG_L1       0.793561   0.880887     roc_auc        0.029005       0.014909   1.128494                 0.029005                0.014909           1.128494            1       True         57
2           LightGBMLarge_BAG_L1       0.787879   0.830884     roc_auc        0.020609       0.010498   1.221966                 0.020609                0.010498           1.221966            1       True         13
3                LightGBM_BAG_L1       0.785985   0.880423     roc_auc        0.010113       0.00

Resultados para 1063: AUC=0.7636, ACC=0.8153, Time=2558.52s
--- Processando Dataset ID: 1068 com AutoGluon ---


...

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          8
Memory Avail:       27.67 GB / 31.36 GB (88.2%)
Disk Space Avail:   89.44 GB / 97.87 GB (91.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.

Problema binário detectado. Usando a métrica: roc_auc


Leaderboard on holdout data (DyStack):
                          model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0          LightGBMLarge_BAG_L1            1.0   0.991940     roc_auc        0.050400       0.017245    1.803341                 0.050400                0.017245           1.803341            1       True         13
1          CatBoost_r177_BAG_L1            1.0   0.999994     roc_auc        0.060704       0.010600   66.995458                 0.060704                0.010600          66.995458            1       True         14
2       RandomForestEntr_BAG_L1            1.0   0.999866     roc_auc        0.073274       0.107906    0.995412                 0.073274                0.107906           0.995412            1       True          6
3     NeuralNetTorch_r22_BAG_L1            1.0   1.000000     roc_auc        0.098025       0.115

Resultados para 1462: AUC=1.0000, ACC=0.9976, Time=3599.51s

Arquivo 'resultados_autogluon_maquina_6.csv' salvo com sucesso!


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [4]:
# --- CONFIGURAÇÃO DA EXECUÇÃO DISTRIBUÍDA ---
# Altere esta variável em cada máquina de 1 a 10
MACHINE_ID = 9
TOTAL_MACHINES = 10
# --- FIM DA CONFIGURAÇÃO ---

# Lista completa dos 30 datasets do CC18
cc18_ids_full = [
    11, 15, 18, 23, 29, 31, 37, 50, 54, 188, 307, 458, 469, 1049,
    1050, 1063, 1068, 1462, 1464, 1468, 1480, 1494, 1501, 1510,
    6332, 23381, 40966, 40975, 40982, 40994
]

# Lógica para dividir a lista de IDs em lotes
chunk_size = math.ceil(len(cc18_ids_full) / TOTAL_MACHINES)
start_index = (MACHINE_ID - 1) * chunk_size
end_index = start_index + chunk_size
ids_to_process = cc18_ids_full[start_index:end_index]

print(f"--- MÁQUINA {MACHINE_ID}/{TOTAL_MACHINES} ---")
print(f"Processando os seguintes datasets: {ids_to_process}")

todos_resultados = []

for oid in ids_to_process:
    try:
        res = run_autogluon_experiment(oid)
        todos_resultados.append(res)
    except Exception as e:
        print(f"!!!!!! ERRO AO PROCESSAR O DATASET {oid}: {e} !!!!!!")
        import traceback
        traceback.print_exc()
        continue

# Salva o arquivo com um nome único para esta máquina
if todos_resultados:
    df_results = pd.DataFrame(todos_resultados)
    output_filename = f"resultados_autogluon_maquina_{MACHINE_ID}.csv"
    df_results.to_csv(output_filename, index=False)
    print(f"\nArquivo '{output_filename}' salvo com sucesso!")
else:
    print("Nenhuma tarefa foi concluída com sucesso nesta máquina.")

/var/tmp/ipykernel_570763/2334902379.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['37579' '34534' '36858' '25451' '34905' '37080' '35522' '25514' '37822'
 '25437' '37441' '47103' '34259' '37371' '23048' '47204' '34554' '47301'
 '34547' '25450' '34752' '38121' '35462' '23055' '35539' '47203' '37501'
 '37938' '34293' '34749' '34154' '37355' '35874' '34093' '38016' '37758'
 '37392' '47203' '25503' '38013' '35753' '37157' '35816' '34714' '37001'
 '36197' '35529' '34583' '37338' '37505' '34554' '36166' '27490' '36058'
 '37915' '34399' '35880' '34585' '34585' '38240' '47104' '34066' '38121'
 '37874' '35816' '34099' '35752' '35794' '47202' '36167' '35811' '47103'
 '36858' '37365' '35754' '37990' '34453' '37998' '85813' '25440' '38120'
 '34367' '25507' '34685' '47204' '25424' '37182' '47105' '34175' '35458'
 '25502' '32360' '25477' '37386' '37822' '37575' '37371' '37970' '25513'
 '47203' '47202' '34154' '47104' '3

--- MÁQUINA 9/10 ---
Processando os seguintes datasets: [6332, 23381, 40966]
--- Processando Dataset ID: 6332 com AutoGluon ---
Problema binário detectado. Usando a métrica: roc_auc


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-06-14 16:41:27,711	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
		Context path: "/home/jupyter/autogluon_models/ds_6332/ds_sub_fit/sub_fit_ho"
(_dystack pid=571120) Running DyStack sub-fit ...
(_dystack pid=571120) Beginning AutoGluon training ... Time limit = 896s
(_dystack pid=571120) AutoGluon will save models to "/home/jupyter/autogluon_models/ds_6332/ds_sub_fit/sub_fit_ho"
(_dystack pid=571120) Train Data Rows:    336
(_dystack pid=571120) Train Data Columns: 37
(_dystack pid=571120) Label Column:       target
(_dystack pid=571120) Problem Type:       binary
(_dystack pid=571120) Preprocessing data ...
(_dystack pid=571120) Selected class <--> label mapping:  class 1 = 1, class 0 = 0
(_dystack pid=571120) Using Feature Generators to prep

(_ray_fit pid=574783) [1000]	valid_set's binary_logloss: 0.405281


(_dystack pid=571120) 	0.8322	 = Validation score   (roc_auc)
(_dystack pid=571120) 	1.5s	 = Training   runtime
(_dystack pid=571120) 	0.08s	 = Validation runtime
(_dystack pid=571120) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 506.39s of the 805.02s of remaining time.
(_dystack pid=571120) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.00%)
(_ray_fit pid=575100) No improvement since epoch 11: early stopping
(_dystack pid=571120) 	0.8261	 = Validation score   (roc_auc)
(_dystack pid=571120) 	7.03s	 = Training   runtime
(_dystack pid=571120) 	0.19s	 = Validation runtime
(_dystack pid=571120) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 496.68s of the 795.31s of remaining time.
(_dystack pid=571120) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.73%)
(_dystack pid=571120) 	0.8801	 =

...

Resultados para 6332: AUC=0.9002, ACC=0.8395, Time=2036.83s
--- Processando Dataset ID: 23381 com AutoGluon ---
Problema binário detectado. Usando a métrica: roc_auc


Leaderboard on holdout data (DyStack):
                           model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetTorch_r22_BAG_L2       0.657609   0.769529     roc_auc        1.147112       0.871885  40.094265                 0.126741                0.137072          14.316578            2       True        107
1      NeuralNetTorch_r86_BAG_L2       0.654891   0.756404     roc_auc        1.156886       0.872258  35.850710                 0.136515                0.137445          10.073024            2       True        119
2            CatBoost_r50_BAG_L2       0.649457   0.772519     roc_auc        1.054071       0.829603  31.201605                 0.033700                0.094790           5.423918            2       True        120
3           LightGBM_r130_BAG_L2       0.649457   0.734648     roc_auc        1.065149       0.79

Resultados para 23381: AUC=0.5669, ACC=0.5667, Time=2604.85s
--- Processando Dataset ID: 40966 com AutoGluon ---
Problema multiclasse detectado (8 classes). Usando a métrica: roc_auc_ovo


Leaderboard on holdout data (DyStack):
                      model  score_holdout  score_val  eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           LightGBM_BAG_L1       1.000000   0.998211  roc_auc_ovo        0.133694       0.113750   31.577831                 0.133694                0.113750          31.577831            1       True          5
1     NeuralNetTorch_BAG_L1       1.000000   0.999616  roc_auc_ovo        0.146459       0.225369   18.349885                 0.146459                0.225369          18.349885            1       True         12
2     ExtraTreesGini_BAG_L1       1.000000   0.999429  roc_auc_ovo        0.165123       0.101962    1.932706                 0.165123                0.101962           1.932706            1       True          9
3         LightGBMXT_BAG_L1       1.000000   0.998714  roc_auc_ovo        0.175255       0.081657   29.347656

Resultados para 40966: AUC=1.0000, ACC=0.9907, Time=3600.29s

Arquivo 'resultados_autogluon_maquina_9.csv' salvo com sucesso!
